In [ ]:
#@title 0. 기본 라이브러리 임포트
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os
import pandas as pd
import h5py
import pickle
from time import time
import shutil
from tensorboardX import SummaryWriter
import math
import time
from torch.utils.data import TensorDataset, DataLoader
from tqdm.auto import tqdm
import pickle

In [ ]:
#@title 1. 유틸리티 함수 정의

def scaled_Laplacian(W):
    '''
    compute \tilde{L}
    '''
    assert W.shape[0] == W.shape[1]
    D = np.diag(np.sum(W, axis=1))
    L = D - W
    lambda_max = eigs(L, k=1, which='LR')[0].real
    return (2 * L) / lambda_max - np.identity(W.shape[0])

def cheb_polynomial(L_tilde, K):
    '''
    compute a list of chebyshev polynomials from T_0 to T_{K-1}
    '''
    N = L_tilde.shape[0]
    cheb_polynomials = [np.identity(N), L_tilde.copy()]
    for i in range(2, K):
        cheb_polynomials.append(2 * L_tilde @ cheb_polynomials[i - 1] - cheb_polynomials[i - 2])
    return cheb_polynomials

def create_multi_scale_sequences(data, hour_len=6, day_len=24, week_len=24, 
                                output_len=24, week_offset=168):
    """
    Create multi-scale input sequences for MST-GCN
    Updated for 1-hour timesteps
    """
    X_hour, X_day, X_week, y = [], [], [], []
    
    # Start from week_offset to ensure we have enough history
    for i in range(week_offset, data.shape[2] - output_len + 1):
        # Recent pattern (last 6 hours)
        hour_slice = data[:, :, i-hour_len:i]
        X_hour.append(hour_slice)
        
        # Daily pattern (last 24 hours)
        day_slice = data[:, :, i-day_len:i]
        X_day.append(day_slice)
        
        # Weekly pattern (same 24 hours from last week)
        week_slice = data[:, :, i-week_offset:i-week_offset+week_len]
        X_week.append(week_slice)
        
        # Target (next 24 hours, DRT_Demand_Prob which is index 1)
        y_slice = data[1, :, i:i+output_len]  # drt_probability is index 1
        y.append(y_slice)
    
    return np.array(X_hour), np.array(X_day), np.array(X_week), np.array(y)

In [ ]:
#@title 2. MST-GCN 모델 아키텍처

class cheb_conv(nn.Module):
    '''
    K-order chebyshev graph convolution
    '''
    def __init__(self, K, cheb_polynomials, in_channels, out_channels):
        super(cheb_conv, self).__init__()
        self.K = K
        self.cheb_polynomials = cheb_polynomials
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.DEVICE = cheb_polynomials[0].device
        
        self.Theta = nn.ParameterList([
            nn.Parameter(torch.FloatTensor(in_channels, out_channels).to(self.DEVICE)) 
            for _ in range(K)
        ])

    def forward(self, x):
        batch_size, num_of_vertices, in_channels, num_of_timesteps = x.shape
        
        outputs = []
        for time_step in range(num_of_timesteps):
            graph_signal = x[:, :, :, time_step]
            output = torch.zeros(batch_size, num_of_vertices, self.out_channels).to(self.DEVICE)
            
            for k in range(self.K):
                T_k = self.cheb_polynomials[k]
                theta_k = self.Theta[k]
                rhs = torch.matmul(T_k, graph_signal)
                output = output + torch.matmul(rhs, theta_k)
                
            outputs.append(output.unsqueeze(-1))
            
        return F.relu(torch.cat(outputs, dim=-1))


class MSTGCN_block(nn.Module):
    def __init__(self, DEVICE, in_channels, K, nb_chev_filter, nb_time_filter, 
                 time_conv_strides, cheb_polynomials, num_of_vertices, num_of_timesteps):
        super(MSTGCN_block, self).__init__()
        
        self.cheb_conv = cheb_conv(K, cheb_polynomials, in_channels, nb_chev_filter)
        self.time_conv = nn.Conv2d(nb_chev_filter, nb_time_filter, 
                                   kernel_size=(1, 3), stride=(1, time_conv_strides), 
                                   padding=(0, 1))
        self.residual_conv = nn.Conv2d(in_channels, nb_time_filter, 
                                       kernel_size=(1, 1), stride=(1, time_conv_strides))
        self.ln = nn.LayerNorm(nb_time_filter)

    def forward(self, x):
        spatial_gcn = self.cheb_conv(x)
        time_conv_output = self.time_conv(spatial_gcn.permute(0, 2, 1, 3))
        time_conv_output = time_conv_output.permute(0, 2, 1, 3)
        
        x_residual = self.residual_conv(x.permute(0, 2, 1, 3))
        x_residual = x_residual.permute(0, 2, 1, 3)
        
        out = F.relu(x_residual + time_conv_output)
        out = out.permute(0, 1, 3, 2)
        out = self.ln(out)
        out = out.permute(0, 1, 3, 2)
        
        return out


class MSTGCN_submodule(nn.Module):
    def __init__(self, DEVICE, nb_block, in_channels, K, nb_chev_filter, nb_time_filter, 
                 time_strides, cheb_polynomials, num_for_predict, len_input, num_of_vertices):
        super(MSTGCN_submodule, self).__init__()
        
        self.BlockList = nn.ModuleList()
        
        self.BlockList.append(
            MSTGCN_block(DEVICE, in_channels, K, nb_chev_filter, nb_time_filter, 
                         time_strides, cheb_polynomials, num_of_vertices, len_input)
        )
        
        for _ in range(nb_block - 1):
            self.BlockList.append(
                MSTGCN_block(DEVICE, nb_time_filter, K, nb_chev_filter, nb_time_filter, 
                             1, cheb_polynomials, num_of_vertices, len_input // time_strides)
            )
        
        self.final_conv = nn.Conv2d(int(len_input / time_strides), num_for_predict, 
                                   kernel_size=(1, nb_time_filter))
        self.W = nn.Parameter(torch.FloatTensor(num_of_vertices, num_for_predict))
        
        self.DEVICE = DEVICE
        self.to(DEVICE)

    def forward(self, x):
        for block in self.BlockList:
            x = block(x)
            
        output = self.final_conv(x.permute(0, 3, 1, 2))
        output = output[:, :, :, 0].permute(0, 2, 1)
        output = output * self.W
        
        return output


class MSTGCN(nn.Module):
    '''
    Multi-Scale Temporal Graph Convolutional Networks
    '''
    def __init__(self, DEVICE, nb_block, in_channels, K, nb_chev_filter, nb_time_filter,
                 time_strides, cheb_polynomials, num_for_predict, num_of_vertices,
                 len_hour, len_day, len_week):
        super(MSTGCN, self).__init__()
        
        self.num_for_predict = num_for_predict
        
        self.hour_module = MSTGCN_submodule(
            DEVICE, nb_block, in_channels, K, nb_chev_filter, nb_time_filter,
            time_strides, cheb_polynomials, num_for_predict, len_hour, num_of_vertices
        )
        
        self.day_module = MSTGCN_submodule(
            DEVICE, nb_block, in_channels, K, nb_chev_filter, nb_time_filter,
            time_strides, cheb_polynomials, num_for_predict, len_day, num_of_vertices
        )
        
        self.week_module = MSTGCN_submodule(
            DEVICE, nb_block, in_channels, K, nb_chev_filter, nb_time_filter,
            time_strides, cheb_polynomials, num_for_predict, len_week, num_of_vertices
        )

    def forward(self, x_hour, x_day, x_week):
        hour_output = self.hour_module(x_hour)
        day_output = self.day_module(x_day)
        week_output = self.week_module(x_week)
        
        output = hour_output + day_output + week_output
        
        return output

In [ ]:
#@title 3. 데이터 로딩 및 전처리 (8개월 통합 데이터)

# Google Colab에서 학습할 때 사용할 코드
# 로컬에서 8개월 데이터를 통합한 dataset.npz 파일 사용

# 🔄 수정: 통합된 8개월 데이터 로딩
base_path = '/content/drive/MyDrive/train_dataset/dataset.npz'
data = np.load(base_path)

feature_matrix = data['feature_matrix']  # (5, N, T) - 4개 입력 + 1개 타겟
stop_ids = data['stop_ids'] 
adj_mx = data['adj_matrix']

print("✅ 통합된 8개월 MST-GCN 데이터 로딩 완료:")
print(f"Feature matrix shape: {feature_matrix.shape}")
print(f"정류장 수: {len(stop_ids)}")
print(f"인접 행렬 shape: {adj_mx.shape}")

# 피처 구성 확인
print("\n피처 구성 (5개):")
print("  [0] normalized_log_boarding_count - Log+Z-score 정규화 수요")
print("  [1] service_availability - 서비스 가용성 (0,1,2)")
print("  [2] is_rest_day - 휴식일 여부 (0,1)")  
print("  [3] normalized_interval - 정규화된 배차간격 [0,1]")
print("  [4] drt_probability - DRT 확률 (타겟)")

# 입력 피처와 타겟 분리
input_features = feature_matrix[:4, :, :]  # 처음 4개가 입력
target_feature = feature_matrix[4, :, :]   # 마지막이 타겟

print(f"\n입력 피처 shape: {input_features.shape}")
print(f"타겟 피처 shape: {target_feature.shape}")

# 인접 행렬 통계
edge_count = np.sum(adj_mx) / 2
avg_degree = np.sum(adj_mx, axis=1).mean()
print(f"\n인접 행렬 통계:")
print(f"총 엣지 수: {edge_count}")
print(f"평균 차수: {avg_degree:.2f}")

# 피처별 통계 출력
for i, feature_name in enumerate(['norm_log_boarding', 'service_avail', 'is_rest_day', 'norm_interval']):
    feat_data = input_features[i]
    print(f"\n{feature_name} 통계:")
    print(f"  범위: [{feat_data.min():.4f}, {feat_data.max():.4f}]")
    print(f"  평균: {feat_data.mean():.4f}")
    print(f"  표준편차: {feat_data.std():.4f}")

In [ ]:
#@title 4. 학습 시퀀스 로딩 (통합된 8개월 데이터에서 직접)

# 데이터 로더(data_preparation/mstgcn_data_loader.py)가
# 이미 생성한 다중 스케일 시퀀스를 통합된 npz 파일에서 직접 로드합니다.
# 이 셀은 기존의 시퀀스 생성 코드를 대체합니다.

# Cell 3에서 로드한 'data' 객체에서 각 배열을 가져옵니다.
X_hour = data['X_hour']
X_day = data['X_day']
X_week = data['X_week']
y = data['y']

print("✅ 사전 처리된 다중 스케일 시퀀스를 통합된 8개월 데이터에서 직접 로드했습니다.")
print("\n--- 로드된 데이터 Shape ---")
print(f"X_hour: {X_hour.shape}")
print(f"X_day:  {X_day.shape}")
print(f"X_week: {X_week.shape}")
print(f"y:      {y.shape}")

print(f"\n📊 8개월 통합 데이터 통계:")
print(f"총 샘플 수: {X_hour.shape[0]}")
print(f"정류장 수: {X_hour.shape[1]}")
print(f"입력 피처 수: {X_hour.shape[2]}")
print(f"예측 길이: {y.shape[2]} 시간")

# 다음 셀에서 사용할 변수들이 올바르게 로드되었는지 확인
assert 'adj_mx' in locals() or 'adj_mx' in globals(), "오류: 'adj_mx'가 이전 셀에서 로드되지 않았습니다."
assert X_hour.shape[2] == 4, f"오류: 입력 피처의 수가 4가 아닙니다. 감지된 수: {X_hour.shape[2]}"

print("\n✅ 데이터 구조 검증 완료. 다음 단계 진행 가능.")

In [ ]:
#@title 5. 하이퍼파라미터 및 설정 정의 (4개 입력 피처)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', DEVICE)

# 학습 하이퍼파라미터
LEARNING_RATE = 0.001
EPOCHS = 100
BATCH_SIZE = 32

# MST-GCN 모델 파라미터
K = 3                  # Chebyshev 차수 : 그래프 컨볼루션 연산 시, 각 노드가 몇 홉(hop) 떨어진 이웃의 정보까지 참고할 것인지를 의미
nb_block = 2           # 블록 수 : 하나의 서브모듈(hour, day, week) 안에 Spatio-Temporal 블록을 몇 개나 쌓을 것인지
nb_chev_filter = 64    # 공간 필터 수 : 그래프 컨볼루션(공간)을 통해 추출할 특징(피처)의 개수
nb_time_filter = 64    # 시간 필터 수 : 시간적 컨볼루션을 통해 추출할 특징(피처)의 개수
time_strides = 1       # 스트라이드 : 시간적 컨볼루션 연산 시, 몇 칸씩 건너뛰며 적용할지 결정(1인 경우 모든 시점 체크)

# 데이터 shape 기반 파라미터(shape에 따라 자동 결정)
num_of_vertices = adj_mx.shape[0]
in_channels = X_hour.shape[2]  # 4개 입력 피처
num_for_predict = y.shape[2]   # 24시간 예측
len_hour = X_hour.shape[3]     # 6시간
len_day = X_day.shape[3]       # 24시간
len_week = X_week.shape[3]     # 24시간

print(f"\n모델 파라미터 (4개 입력 피처 기준):")
print(f"정류장 수: {num_of_vertices}")
print(f"입력 피처 수: {in_channels} (normalized_log_boarding_count, service_availability, is_rest_day, normalized_interval)")
print(f"예측 시간 길이: {num_for_predict}")
print(f"Hour 스케일 길이: {len_hour}")
print(f"Day 스케일 길이: {len_day}")
print(f"Week 스케일 길이: {len_week}")

# 모델 입력 검증
assert in_channels == 4, f"입력 피처 수가 4개가 아닙니다: {in_channels}"
print(f"\n✅ 4개 입력 피처 확인 완료")

# 각 피처의 의미 설명
print(f"\n입력 피처 구성:")
print(f"  피처 0: normalized_log_boarding_count - Log+Z-score 정규화된 수요 (연속값)")
print(f"  피처 1: service_availability - 서비스 가용성 (0=비운행, 1=시간외, 2=시간내)")
print(f"  피처 2: is_rest_day - 휴식일 여부 (0=평일, 1=휴일)")
print(f"  피처 3: normalized_interval - 정규화된 배차간격 (0~1)")

# 데이터 범위 검증
print(f"\n데이터 범위 검증:")
for i, feature_name in enumerate(['norm_log_boarding', 'service_avail', 'is_rest_day', 'norm_interval']):
    feat_data = X_hour[:, :, i, :]  # 각 피처의 데이터
    print(f"  {feature_name}: [{feat_data.min():.4f}, {feat_data.max():.4f}]")

In [ ]:
#@title 6. 데이터 로더 생성 (🚨 Data Leakage 방지 - 시간 순서 기반 분할)

# numpy 배열을 torch 텐서로 변환
X_hour_tensor = torch.from_numpy(X_hour).type(torch.FloatTensor)
X_day_tensor = torch.from_numpy(X_day).type(torch.FloatTensor)
X_week_tensor = torch.from_numpy(X_week).type(torch.FloatTensor)
y_tensor = torch.from_numpy(y).type(torch.FloatTensor)

# Custom Dataset for multi-scale inputs
class MultiScaleDataset(torch.utils.data.Dataset):
    def __init__(self, X_hour, X_day, X_week, y):
        self.X_hour = X_hour
        self.X_day = X_day
        self.X_week = X_week
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X_hour[idx], self.X_day[idx], self.X_week[idx], self.y[idx]

# 전체 데이터셋 생성
dataset = MultiScaleDataset(X_hour_tensor, X_day_tensor, X_week_tensor, y_tensor)

# 🚨 중요: 시간 순서 기반 분할 (Data Leakage 방지)
# random_split 대신 시간 순서를 보장하는 분할 사용
dataset_size = len(dataset)
train_end = int(dataset_size * 0.7)
val_end = int(dataset_size * 0.85)

# 시간 순서대로 인덱스 생성
train_indices = list(range(0, train_end))
val_indices = list(range(train_end, val_end))
test_indices = list(range(val_end, dataset_size))

print(f"🚨 Data Leakage 방지를 위한 시간 순서 기반 분할:")
print(f"   Train: 샘플 [0:{train_end}] → 초기 70% 시간대 ({len(train_indices)} 샘플)")
print(f"   Val:   샘플 [{train_end}:{val_end}] → 중간 15% 시간대 ({len(val_indices)} 샘플)")
print(f"   Test:  샘플 [{val_end}:{dataset_size}] → 마지막 15% 시간대 ({len(test_indices)} 샘플)")

# Subset으로 데이터셋 분할
from torch.utils.data import Subset
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)

# DataLoader 생성 (Train만 셔플, Val/Test는 시간 순서 유지)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"\n📊 최종 데이터셋 크기:")
print(f"전체: {len(dataset)} 샘플 (8개월 통합)")
print(f"학습: {len(train_dataset)} 샘플")
print(f"검증: {len(val_dataset)} 샘플")
print(f"테스트: {len(test_dataset)} 샘플")

print(f"\n✅ 시계열 특성을 보존한 신뢰할 수 있는 데이터 분할 완료")

In [ ]:
#@title 7. 모델, 손실 함수, 옵티마이저 초기화

# 체비셰프 다항식 계산
L_tilde = scaled_Laplacian(adj_mx)
cheb_polynomials = [torch.from_numpy(i).type(torch.FloatTensor).to(DEVICE) for i in cheb_polynomial(L_tilde, K)]

# 모델 인스턴스 생성
model = MSTGCN(
    DEVICE, nb_block, in_channels, K, nb_chev_filter, nb_time_filter,
    time_strides, cheb_polynomials, num_for_predict, num_of_vertices,
    len_hour, len_day, len_week
)
model.to(DEVICE)

# 손실 함수: MSE (DRT 확률 예측용)
loss_fn = nn.MSELoss()

# 옵티마이저
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 모델 가중치 초기화
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
    else:
        nn.init.uniform_(p)

print("MST-GCN 모델 구조:")
print(model)

In [ ]:
#@title 8. 모델 학습 및 검증 루프

print("\n학습 시작...")
best_val_loss = float('inf')
best_epoch = -1
history = {'train_loss': [], 'val_loss': []}

# 학습 데이터 통계 (정규화용)
train_indices = train_dataset.indices
X_hour_train = X_hour_tensor[train_indices]
X_day_train = X_day_tensor[train_indices]
X_week_train = X_week_tensor[train_indices]

# 각 스케일별 평균과 표준편차 계산
hour_mean = X_hour_train.mean()
hour_std = X_hour_train.std()
day_mean = X_day_train.mean()
day_std = X_day_train.std()
week_mean = X_week_train.mean()
week_std = X_week_train.std()

print(f"정규화 통계:")
print(f"Hour - Mean: {hour_mean:.4f}, Std: {hour_std:.4f}")
print(f"Day - Mean: {day_mean:.4f}, Std: {day_std:.4f}")
print(f"Week - Mean: {week_mean:.4f}, Std: {week_std:.4f}")

for epoch in range(EPOCHS):
    start_time = time.time()
    
    # 학습 단계
    model.train()
    total_train_loss = 0
    for x_hour, x_day, x_week, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]"):
        x_hour, x_day, x_week, y_batch = x_hour.to(DEVICE), x_day.to(DEVICE), x_week.to(DEVICE), y_batch.to(DEVICE)
        
        # 순전파
        output = model(x_hour, x_day, x_week)
        loss = loss_fn(output, y_batch)
        
        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item()
    
    avg_train_loss = total_train_loss / len(train_loader)
    history['train_loss'].append(avg_train_loss)
    
    # 검증 단계
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for x_hour, x_day, x_week, y_val in tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Val]"):
            x_hour, x_day, x_week, y_val = x_hour.to(DEVICE), x_day.to(DEVICE), x_week.to(DEVICE), y_val.to(DEVICE)
            output = model(x_hour, x_day, x_week)
            loss = loss_fn(output, y_val)
            total_val_loss += loss.item()
    
    avg_val_loss = total_val_loss / len(val_loader)
    history['val_loss'].append(avg_val_loss)
    
    end_time = time.time()
    epoch_time = end_time - start_time
    
    print(f"Epoch [{epoch+1}/{EPOCHS}] | Train Loss: {avg_train_loss:.6f} | Val Loss: {avg_val_loss:.6f} | Time: {epoch_time:.2f}s")
    
    # 최고 성능 모델 저장
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_epoch = epoch
        torch.save(model.state_dict(), 'best_mstgcn_model.pth')
        print(f"-> Best model saved at epoch {epoch+1} with validation loss {best_val_loss:.6f}")

print("\n학습 완료!")
print(f"Best validation loss {best_val_loss:.6f} at epoch {best_epoch+1}")

In [ ]:
#@title 9. TorchServe 배포를 위한 모델 및 아티팩트 저장 (8개월 학습 모델)

save_dir = '/content/drive/MyDrive/MSTGCN_results_8months'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 1. 전체 모델 저장 (.pt 파일)
model.load_state_dict(torch.load('best_mstgcn_model.pth'))
model.eval()
full_model_path = os.path.join(save_dir, 'mstgcn_model_full_8months.pt')
torch.save(model, full_model_path)

# 2. 체비셰프 다항식 저장
cheb_path = os.path.join(save_dir, 'cheb_polynomials.pt')
cheb_cpu = [poly.cpu() for poly in cheb_polynomials]
torch.save(cheb_cpu, cheb_path)

# 3. 인접 행렬 저장
adj_path = os.path.join(save_dir, 'adjacency_matrix.npy')
np.save(adj_path, adj_mx)

# 4. 정규화 통계 저장
stats_path = os.path.join(save_dir, 'normalization_stats.npz')
np.savez(stats_path, 
         hour_mean=hour_mean.numpy(), hour_std=hour_std.numpy(),
         day_mean=day_mean.numpy(), day_std=day_std.numpy(),
         week_mean=week_mean.numpy(), week_std=week_std.numpy())

# 5. 정류장 매핑 정보 저장 (8개월 통합 데이터 기준)
stop_mapping_path = os.path.join(save_dir, 'stop_mapping_8months.pkl')
stop_mapping = {
    'stop_ids': stop_ids.tolist(),
    'num_stops': len(stop_ids),
    'training_period': '2024-11 to 2025-06 (8 months)',
    'common_stops_only': True
}
with open(stop_mapping_path, 'wb') as f:
    pickle.dump(stop_mapping, f)

# 6. 모델 설정 저장 (8개월 데이터 기준)
config_path = os.path.join(save_dir, 'model_config_8months.pkl')
model_config = {
    'nb_block': nb_block,
    'in_channels': in_channels,
    'K': K,
    'nb_chev_filter': nb_chev_filter,
    'nb_time_filter': nb_time_filter,
    'time_strides': time_strides,
    'num_for_predict': num_for_predict,
    'num_of_vertices': num_of_vertices,
    'len_hour': len_hour,
    'len_day': len_day,
    'len_week': len_week,
    'device': str(DEVICE),
    'training_samples': len(dataset),
    'training_period': '8 months (2024-11 to 2025-06)',
    'data_leakage_prevented': True,
    'temporal_split': True
}
with open(config_path, 'wb') as f:
    pickle.dump(model_config, f)

# 7. 학습 히스토리 저장
history_path = os.path.join(save_dir, 'training_history_8months.pkl')
enhanced_history = {
    'train_loss': history['train_loss'],
    'val_loss': history['val_loss'],
    'best_epoch': best_epoch,
    'best_val_loss': best_val_loss,
    'total_epochs': EPOCHS,
    'batch_size': BATCH_SIZE,
    'learning_rate': LEARNING_RATE,
    'training_samples': len(train_dataset),
    'validation_samples': len(val_dataset),
    'test_samples': len(test_dataset)
}
with open(history_path, 'wb') as f:
    pickle.dump(enhanced_history, f)

print("\n✅ TorchServe 배포용 파일 저장 완료 (8개월 학습 모델):")
print(f"   - 전체 모델: {full_model_path}")
print(f"   - 체비셰프 다항식: {cheb_path}")
print(f"   - 인접 행렬: {adj_path}")
print(f"   - 정규화 통계: {stats_path}")
print(f"   - 정류장 매핑: {stop_mapping_path}")
print(f"   - 모델 설정: {config_path}")
print(f"   - 학습 히스토리: {history_path}")
print(f"\n🎯 특징:")
print(f"   - 8개월 연속 데이터로 학습")
print(f"   - Data Leakage 방지 적용")
print(f"   - 시간 순서 기반 분할")
print(f"   - 총 {len(dataset)} 샘플 학습")

In [ ]:
#@title 10. 테스트 셋에서 최종 성능 평가

model.eval()
test_loss = 0
predictions = []
actuals = []

with torch.no_grad():
    for x_hour, x_day, x_week, y_test in tqdm(test_loader, desc="Test Evaluation"):
        x_hour, x_day, x_week, y_test = x_hour.to(DEVICE), x_day.to(DEVICE), x_week.to(DEVICE), y_test.to(DEVICE)
        output = model(x_hour, x_day, x_week)
        loss = loss_fn(output, y_test)
        test_loss += loss.item()
        
        predictions.append(output.cpu().numpy())
        actuals.append(y_test.cpu().numpy())

avg_test_loss = test_loss / len(test_loader)
print(f"\n테스트 셋 MSE Loss: {avg_test_loss:.6f}")
print(f"테스트 셋 RMSE: {np.sqrt(avg_test_loss):.6f}")

# 예측 결과 저장
predictions = np.concatenate(predictions, axis=0)
actuals = np.concatenate(actuals, axis=0)
test_results_path = os.path.join(save_dir, 'test_predictions.npz')
np.savez(test_results_path, predictions=predictions, actuals=actuals)
print(f"테스트 예측 결과 저장: {test_results_path}")

print("\n🎉 MST-GCN 학습 및 저장 완료!")

In [ ]:
#@title 1. (추론) 저장된 모델 및 아티팩트 로드

save_dir = '/content/drive/MyDrive/MSTGCN_results'

# 1. 전체 모델 로드
model_path = os.path.join(save_dir, 'mstgcn_model_full.pt')
model = torch.load(model_path, map_location='cuda' if torch.cuda.is_available() else 'cpu')
model.eval()
print("✅ 모델 로드 완료")

# 2. 체비셰프 다항식 로드
cheb_path = os.path.join(save_dir, 'cheb_polynomials.pt')
cheb_polynomials = torch.load(cheb_path)
print("✅ 체비셰프 다항식 로드 완료")

# 3. 정규화 통계 로드
stats_path = os.path.join(save_dir, 'normalization_stats.npz')
stats = np.load(stats_path)
hour_mean, hour_std = stats['hour_mean'], stats['hour_std']
day_mean, day_std = stats['day_mean'], stats['day_std']
week_mean, week_std = stats['week_mean'], stats['week_std']
print("✅ 정규화 통계 로드 완료")

# 4. 정류장 매핑 정보 로드
stop_mapping_path = os.path.join(save_dir, 'stop_mapping.pkl')
with open(stop_mapping_path, 'rb') as f:
    stop_mapping = pickle.load(f)
stop_ids = stop_mapping['stop_ids']
stop_info = stop_mapping['stop_info']
print(f"✅ 정류장 매핑 로드 완료 ({len(stop_ids)}개 정류장)")

# 5. 모델 설정 로드
config_path = os.path.join(save_dir, 'model_config.pkl')
with open(config_path, 'rb') as f:
    model_config = pickle.load(f)
print("✅ 모델 설정 로드 완료")

In [ ]:
#@title 2. (추론) 추론 함수 정의

def prepare_inference_data(current_time, feature_matrix, time_index):
    """
    현재 시간을 기준으로 MST-GCN 입력 데이터 준비
    
    Parameters:
    current_time: 현재 시간 (datetime)
    feature_matrix: 전체 피처 행렬 (F, N, T)
    time_index: 시간 인덱스
    
    Returns:
    x_hour, x_day, x_week 텐서
    """
    # 현재 시간의 인덱스 찾기
    current_idx = time_index.get_loc(current_time)
    
    # 최근 6시간 데이터
    hour_start = current_idx - 6
    x_hour = feature_matrix[:, :, hour_start:current_idx]
    
    # 과거 24시간 데이터
    day_start = current_idx - 24
    x_day = feature_matrix[:, :, day_start:current_idx]
    
    # 1주일 전 24시간 데이터
    week_start = current_idx - 168
    week_end = week_start + 24
    
    #.....
